In [7]:
import cv2
import pandas as pd
import time
from datetime import datetime 
import csv
import os
import yagmail





def data(csv_file):
    with open(csv_file, 'r') as file:
        # Create a CSV reader
        csv_reader = csv.reader(file)
        
        # Read the header (column names)
        header = next(csv_reader, None)
        
        # Check if there is at least one more row (data)
        return any(row for row in csv_reader)


def main():
    
    csv_file_path = 'C:/Users/Lenovo/Desktop/SmartCity/movements.csv'

    video_file_path = 'C:/Users/Lenovo/Desktop/SmartCity/file.avi'


    duration = 30

    start = time.time()


    static_back = None
  

    motion_list = [ None, None ] 
  

    date = [] 
  

    df = pd.DataFrame(columns = ["Start", "End"]) 
  

    video = cv2.VideoCapture(0) 
  
    if (video.isOpened() == False):  
        print("Error reading video file") 
  
    # We need to set resolutions. 

    frame_width = int(video.get(3)) 
    frame_height = int(video.get(4)) 
   
    size = (frame_width, frame_height) 


    result = cv2.VideoWriter('file.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size)    


    while True: 
  
        check, frame = video.read() 

        motion = 0
  
 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
  

        gray = cv2.GaussianBlur(gray, (21, 21), 0) 
  

        if static_back is None: 
            static_back = gray 
            continue
  

        diff_frame = cv2.absdiff(static_back, gray) 
  
 
        thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1] 
        thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2) 
  
 
        cnts,_ = cv2.findContours(thresh_frame.copy(),  
                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
  
        for contour in cnts: 
            if cv2.contourArea(contour) < 10000: 
                continue
            motion = 1
          
            (x, y, w, h) = cv2.boundingRect(contour) 
    
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3) 
       
    
            result.write(frame) 
       


        motion_list[len(motion_list):] = [motion]
    
  
        motion_list = motion_list[-2:] 
  
  
        if motion_list[-1] == 1 and motion_list[-2] == 0: 
     
            date[len(date):] = [datetime.now()] 
     
       
  
        if motion_list[-1] == 0 and motion_list[-2] == 1: 

            date[len(date):] = [datetime.now()]  
        
         
    
        cv2.imshow("Color Frame", frame) 
    
    
    
    
   
        if cv2.waitKey(1) & 0xFF == ord('q'):   
            if motion == 1: 
           
                date[len(date):] = [datetime.now()]
            break
    
        if time.time()-start>duration:
            if motion == 1: 
           
                date[len(date):] = [datetime.now()]
            break
    
     
        
  
    
    for i in range(0, len(date), 2): 
    
        new_item = [date[i],date[i+1]] 
        df.loc[len(df)] = new_item
  
    #saving video
    df.to_csv("movements.csv") 
  
    video.release() 
    result.release()   

    cv2.destroyAllWindows() 
    print("The video was successfully saved") 

    time.sleep(5)
    if data(csv_file_path):
        print(f"The CSV file '{csv_file_path}' has more than just column names.")
        print("An email was sent")
        email_address = ''
        email_password = ''
        yag = yagmail.SMTP(email_address, email_password)
        to = ''
        subject = 'Motion detected'
        body = 'Video and CSV file of motion'
        att =['C:/Users/Lenovo/Desktop/SmartCity/movements.csv','C:/Users/Lenovo/Desktop/SmartCity/file.avi']
        yag.send(to, subject, body,att)
        yag.close()
        time.sleep(5)
        os.remove(csv_file_path)
        os.remove(video_file_path)
    else:
        print(f"The CSV file '{csv_file_path}' only contains column names.")
        time.sleep(3)
        os.remove(csv_file_path)
        print("The CSV file was deleted ")
        os.remove(video_file_path)
        print("The Video file was deleted")
    
#     Recursive call of main function  
#     time.sleep(5)
#     main()
        
if __name__ == "__main__":
    # Call the main function
    main()    
    


The video was successfully saved
The CSV file 'C:/Users/Lenovo/Desktop/SmartCity/movements.csv' has more than just column names.
An email was sent
